In [ ]:
import numpy as np
import os
import dlib
import albumentations as A
import cv2
from imutils import face_utils
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.python.data import AUTOTUNE
import yaml
import wandb

In [ ]:
wandb.login(key=open('../secrets/wandb_key.txt', 'r').read(), relogin=True)
config = yaml.safe_load(open('../config.yaml', 'r'))

## Create DS

### Funcs

In [ ]:
image_size = 192
data_dir = '../data/'

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100


In [ ]:
from ..src.dataset import create_dataset, compress_splits

### Create dataset from scratch

In [ ]:
with wandb.init(project=config['wandb']['project'],
           name='Dataset',
           config=config):
    i, k = create_dataset(dataset_path=data_dir, image_size=image_size)
    compress_splits(i,k, '../data/')    

### Load ds

In [ ]:
train_dataset, test_dataset = fetch_ds(config, 'train')

## Model training


In [ ]:
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from wandb.keras import WandbCallback

from src.data_tests import pass_tests_before_fitting
from src.model import compile_model
from src.dataset import fetch_ds

In [ ]:
with wandb.init(project=config['wandb']['project'],
           name=config['wandb']['name'],
           config=config):
    model = compile_model(
        input_shape=config['img_shape'], output_shape=config['kp_shape'])

    callbacks = [EarlyStopping(**config['callbacks']['EarlyStopping']),
                ReduceLROnPlateau(**config['callbacks']['ReduceLROnPlateau']),
                WandbCallback(**config['callbacks']['WandbCallback'])]

    # data tests (pre-fitting)
    pass_tests_before_fitting(
        data=train_dataset, img_shape=config['img_shape'], keypoint_shape=config['kp_shape'])
    pass_tests_before_fitting(
        data=test_dataset, img_shape=config['img_shape'],  keypoint_shape=config['kp_shape'])

    # training
    history = model.fit(
        train_dataset, epochs=config['train']['epochs'], validation_data=test_dataset, callbacks=callbacks)


## Model optimization

In [ ]:
import tempfile
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from wandb.keras import WandbCallback


from src.data_tests import pass_tests_before_fitting
from src.model import compile_model
from src.dataset import fetch_ds


In [ ]:
from ..optimization import optimize

In [ ]:
optimize(config, model)